<a href="https://colab.research.google.com/github/TharinsaMudalige/Neuron-Brain_Tumor_Detection_Classification_with_XAI/blob/Detection-Classification-VIT/Tumor_Segmentation_using_U_NET_and_Detection_and_Classification_using_ViT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing Libraries

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, Model
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


U- Net Model for Brain Tumor Segmentation

In [ ]:
def unet_model(input_shape=(256, 256, 1)):

    # Builds a U-Net model for image segmentation.
    # U-Net is a convolutional neural network architecture designed for biomedical image segmentation.

    # Input layer for the U-Net
    inputs = tf.keras.Input(input_shape)  # Input tensor for the model

    # Encoder (Downsampling Path)

    # Block 1: First convolutional block
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)  # Convolution layer 1
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)      # Convolution layer 2
    p1 = layers.MaxPooling2D((2, 2))(c1)  # Max pooling layer reduces dimensions by 2x

    # Block 2: Second convolutional block
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)  # Convolution layer 1
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)  # Convolution layer 2
    p2 = layers.MaxPooling2D((2, 2))(c2)  # Max pooling layer reduces dimensions further

    # Bottleneck (Bridge between Encoder and Decoder)
    # Block 3: Bottleneck convolutional block
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)  # Convolution layer 1
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)  # Convolution layer 2

    # Decoder (Upsampling Path)

    # Block 4: First upsampling and convolutional block
    u2 = layers.UpSampling2D((2, 2))(c3)  # Upsampling layer: Doubles the spatial dimensions
    u2 = layers.Concatenate()([u2, c2])   # Skip connection: Concatenate with corresponding encoder block
    c4 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u2)  # Convolution layer 1
    c4 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c4)  # Convolution layer 2

    # Block 5: Second upsampling and convolutional block
    u1 = layers.UpSampling2D((2, 2))(c4)  # Upsampling layer: Doubles the spatial dimensions
    u1 = layers.Concatenate()([u1, c1])   # Skip connection: Concatenate with corresponding encoder block
    c5 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u1)  # Convolution layer 1
    c5 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c5)  # Convolution layer 2

    # Output layer
    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c5)  # Sigmoid activation for binary segmentation

    # Return the U-Net model
    return Model(inputs, outputs, name="U-Net")

# Instantiate U-Net model
unet = unet_model()

# Compile the model
unet.compile(optimizer="adam",                # Adam optimizer for faster convergence
             loss="binary_crossentropy",      # Binary cross-entropy for pixel-wise binary classification
             metrics=["accuracy"])            # Track accuracy during training

# Print model summary
unet.summary()  # Display the model architecture
